In [ ]:
import pandas as pd
import numpy as np
import bigframes.pandas as bf
from google.cloud import bigquery

In [ ]:
bf.options.bigquery.project = 'credit-business-on-demand-9fd5'

In [ ]:
base = pd.read_csv('base_concorrencia_oferta_interagiu_ativo.csv')

In [ ]:
base['Rating'] = pd.to_numeric(base['Rating'].str.replace(',','.'))

In [ ]:
lista_base_filled = []

In [ ]:
for nome_banco in ['Banco Inter', 'Banco do Brasil', 'Bradesco',
       'C6 Bank', 'Caixa', 'Cooperativas',
       'Itaú', 'Mercado Pago', 'Nubank', 'Pagseguro',
       'Santander']:

    print(nome_banco)
    
    base_santander = base[base['IF_adj'] == nome_banco]
    
    base_santander_tofill = base_santander[(base_santander['prazo_scr'].isna())&(base_santander['taxa_scr'].isna())].drop(columns=['prazo_scr'])
    
    base_santander_filled_1 = base_santander[(~base_santander['prazo_scr'].isna())]
    
    base_santander_cross_1 = pd.merge(base_santander_tofill,base_santander_filled_1,how='cross',suffixes=('_tofill','_filled'))
    
    base_santander_cross_1['month_year_same'] = np.where(base_santander_cross_1['month_year_tofill'] == base_santander_cross_1['month_year_filled'],1,0)
    base_santander_cross_1['PerfilSocietario_same'] = np.where(base_santander_cross_1['PerfilSocietario_tofill'] == base_santander_cross_1['PerfilSocietario_filled'],1,0)
    base_santander_cross_1['GrupoCNAE_same'] = np.where(base_santander_cross_1['GrupoCNAE_tofill'] == base_santander_cross_1['GrupoCNAE_filled'],1,0)
    base_santander_cross_1['CustomerUF_same'] = np.where(base_santander_cross_1['CustomerUF_tofill'] == base_santander_cross_1['CustomerUF_filled'],1,0)
    base_santander_cross_1['TierLimite_same'] = np.where(base_santander_cross_1['TierLimite_tofill'] == base_santander_cross_1['TierLimite_filled'],1,0)
    
    base_santander_cross_1['soma_same'] = base_santander_cross_1['month_year_same'] + base_santander_cross_1['PerfilSocietario_same'] + base_santander_cross_1['GrupoCNAE_same'] + base_santander_cross_1['CustomerUF_same'] + base_santander_cross_1['TierLimite_same']
    
    base_santander_cross_1_filled = base_santander_cross_1.loc[base_santander_cross_1.groupby(['RootDocumentNumber_tofill','month_year_tofill'])['soma_same'].idxmax()]
    
    base_santander_cross_1_filled = base_santander_cross_1_filled[base_santander_cross_1_filled['soma_same'] >= 4]
    
    base_santander_cross_1_filled.rename(columns={'RootDocumentNumber_tofill':'RootDocumentNumber',
                                              'month_year_tofill':'month_year','prazo_scr_filled':'prazo_scr'},inplace=True)
    
    base_santander_tofill_complete = base_santander_tofill.merge(base_santander_cross_1_filled[['RootDocumentNumber','month_year','prazo_scr']],how='left',on=['RootDocumentNumber','month_year'])
    
    base_santander_complete_1 = pd.concat([base_santander_tofill_complete,base_santander_filled_1])    
    
    base_santander_tofill_2 = base_santander_complete_1[base_santander_complete_1['taxa_scr'].isna()].drop(columns=['taxa_scr'])
    base_santander_filled_2 = base_santander_complete_1[~base_santander_complete_1['taxa_scr'].isna()]
    
    base_santander_cross_2 = pd.merge(base_santander_tofill_2,base_santander_filled_2,how='cross',suffixes=('_tofill','_filled'))
    
    base_santander_cross_2['prazo_scr_same'] = np.where(base_santander_cross_2['prazo_scr_tofill'] == base_santander_cross_2['prazo_scr_filled'],2,0)
    base_santander_cross_2['month_year_same'] = np.where(base_santander_cross_2['month_year_tofill'] == base_santander_cross_2['month_year_filled'],1,0)
    base_santander_cross_2['PerfilSocietario_same'] = np.where(base_santander_cross_2['PerfilSocietario_tofill'] == base_santander_cross_2['PerfilSocietario_filled'],1,0)
    base_santander_cross_2['GrupoCNAE_same'] = np.where(base_santander_cross_2['GrupoCNAE_tofill'] == base_santander_cross_2['GrupoCNAE_filled'],1,0)
    base_santander_cross_2['CustomerUF_same'] = np.where(base_santander_cross_2['CustomerUF_tofill'] == base_santander_cross_2['CustomerUF_filled'],1,0)
    base_santander_cross_2['TierLimite_same'] = np.where(base_santander_cross_2['TierLimite_tofill'] == base_santander_cross_2['TierLimite_filled'],1,0)
    
    base_santander_cross_2['soma_same'] = base_santander_cross_2['prazo_scr_same'] + base_santander_cross_2['month_year_same'] + base_santander_cross_2['PerfilSocietario_same'] + base_santander_cross_2['GrupoCNAE_same'] + base_santander_cross_2['CustomerUF_same'] + base_santander_cross_2['TierLimite_same']
    
    base_santander_cross_2_filled = base_santander_cross_2.loc[base_santander_cross_2.groupby(['RootDocumentNumber_tofill','month_year_tofill'])['soma_same'].idxmax()]
    
    base_santander_cross_2_filled = base_santander_cross_2_filled[base_santander_cross_2_filled['soma_same'] >= 5]
    
    base_santander_cross_2_filled.rename(columns={'RootDocumentNumber_tofill':'RootDocumentNumber',
                                              'month_year_tofill':'month_year',
                                              'taxa_scr_filled':'taxa_scr'},inplace=True)
    
    base_santander_tofill_complete_2 = base_santander_tofill_2.merge(base_santander_cross_2_filled[['RootDocumentNumber','month_year','taxa_scr']],how='left',on=['RootDocumentNumber','month_year'])
    
    base_santander_complete_2 = pd.concat([base_santander_tofill_complete_2,base_santander_filled_2])
    
    base_santander_complete_2['filled'] = 0
    base_santander_complete_2.loc[base_santander_complete_2['diferenca_taxa'].isna(),'filled'] = 1
    
    base_santander_complete_2 = base_santander_complete_2[['IF_adj', 'RootDocumentNumber', 'month_year', 'desembolso_scr',
       'limite_stone', 'prazo_stone', 'taxa_stone', 'diferenca_limite',
       'diferenca_prazo', 'diferenca_taxa', 'prazo_scr',
       'taxa_scr', 'filled']]
    
    base_santander_complete_2['taxa_stone'] = base_santander_complete_2['taxa_stone'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    base_santander_complete_2['taxa_scr'] = base_santander_complete_2['taxa_scr'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
    
    base_santander_complete_2['diferenca_taxa'] = base_santander_complete_2['taxa_stone'] - base_santander_complete_2['taxa_scr']
    base_santander_complete_2['diferenca_prazo'] = base_santander_complete_2['prazo_stone'] - base_santander_complete_2['prazo_scr']
        
    lista_base_filled.append(base_santander_complete_2)
    

In [ ]:
pd_base_filled = pd.concat(lista_base_filled)

In [ ]:
#%% Exportacao
client = bigquery.Client(project = 'dataplatform-nonprd')

table_name = 'credit_negocios.tbl_base_principalidade_scr_diferenca_condicoes_id_filled'

job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
job = client.load_table_from_dataframe(pd_base_filled, table_name, job_config = job_config)